Universidad del Valle de Guatemala   <br> 
Facultad de Ingeniería    <br>
Data Science    <br>
Andrés Paiz Caniz <b>191142</b> <br>
René Ventura <b></b> 
</p>

<div align="center"><b>
Laboratorio #4 </h1>
    

Se importan las librerias que pensamos utilizar durante el lab.

In [51]:
import numpy as np
import pandas as pd
import seaborn as sns 
import re 

import matplotlib.pyplot as plt 
import nltk 
from wordcloud import WordCloud 
from nltk import word_tokenize 
from nltk.corpus import stopwords 
from nltk.util import ngrams 

import unicodedata 
import string

from collections import Counter, defaultdict 
import zipfile
import random 
from nltk.probability import ConditionalFreqDist




Se carga el archivo train 


Observamos los datos que se cargaron, tienen 5 columnas correspondiendo a:

id: El identificador del tweet 

keyword: una palabra clave del tweet, puede estar en blanco 

location: la ubicación desde donde fue enviado el tweet 

text: El texto del tweet 

target: La etiqueta de clasificación que especifica si el tweet se trata de un desastre real (1) o no (0). 
 

In [39]:
df = pd.read_csv("train lab4.csv")
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


Se puede observar que no todas las columnas son objetivos de limpieza.

In [40]:
df["location"] = df['location'].str.lower()
df["text"] = df['text'].str.lower()
df

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this #earthquake m...,1
1,4,NaN,NaN,forest fire near la ronge sask. canada,1
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,just got sent this photo from ruby #alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @thetawniest the out of control w...,1
7610,10871,NaN,NaN,m1.94 [01:04 utc]?5km s of volcano hawaii. htt...,1
7611,10872,NaN,NaN,police investigating after an e-bike collided ...,1


No se cambian todas las columnas ya que solo location y text tienen mayusculas.

Procedemos a quitar los caracteres especiales (#,@,& y apostrofes)

In [41]:
df["text"] = df["text"].str.replace('[#,@,&]','')
df["location"] = df["location"].str.replace('[#,@,&]','')
df.replace('\'','', regex=True, inplace=True)
df


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16956\3111268107.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace('[#,@,&]','')


,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask. canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,aria_ahrary thetawniest the out of control wil...,1
7610,10871,NaN,NaN,m1.94 [01:04 utc]?5km s of volcano hawaii. htt...,1
7611,10872,NaN,NaN,police investigating after an e-bike collided ...,1


Quitamos los urls 

In [45]:
df["text"] = df["text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
df["text"] = df["text"].str.replace(r'\s*http?://\S+(\s+|$)', ' ').str.strip()
df

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16956\2837192449.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16956\2837192449.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r'\s*http?://\S+(\s+|$)', ' ').str.strip()


,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,aria ahrary thetawniest the out of control wil...,1
7610,10871,NaN,NaN,m1 94 01 04 utc 5km s of volcano hawaii,1
7611,10872,NaN,NaN,police investigating after an e bike collided ...,1


Quitamos emojis al igual que caracteres especiales restantes ( incluyendo punctuacion )

In [50]:
df['text'] = df['text'].str.replace('[^A-Za-z0-9]', ' ', flags=re.UNICODE)
df['location'] = df['location'].str.replace('[^A-Za-z0-9]', ' ', flags=re.UNICODE)
df

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16956\64221057.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^A-Za-z0-9]', ' ', flags=re.UNICODE)


,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,aria ahrary thetawniest the out of control wil...,1
7610,10871,NaN,NaN,m1 94 01 04 utc 5km s of volcano hawaii,1
7611,10872,NaN,NaN,police investigating after an e bike collided ...,1


Eliminamos las stopwords utilizando nltk para descargar la lista de stopwords a eliminar


In [53]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding bridge collapse nearb...,1
7609,10870,NaN,NaN,aria ahrary thetawniest control wild fires cal...,1
7610,10871,NaN,NaN,m1 94 01 04 utc 5km volcano hawaii,1
7611,10872,NaN,NaN,police investigating e bike collided car littl...,1


Eliminamos numeros inecesarios.

,id,keyword,location,text,target
14,20,NaN,NaN,damage school bus 80 multi car crash breaking,1
